# Advent of Code 2019
## [Day 7: Amplification Circuit](https://adventofcode.com/2019/day/7)

In [1]:
from pathlib import Path
import sys
if str(Path('../..').resolve()) not in sys.path:
    sys.path.insert(0, str(Path('../..').resolve()))

from intcode import Intcode

In [2]:
from itertools import permutations
from queue import SimpleQueue
from threading import Thread

In [3]:
with open("input.txt") as f:
    amp_program = [*map(int, f.read().split(","))]
amp_program[:10]

[3, 8, 1001, 8, 10, 8, 105, 1, 0, 0]

In [4]:
cpu = Intcode(amp_program, input=[4,0])
cpu.run()

<Intcode pc:114/519 cycles:8 state:Halted>

In [5]:
cpu.output_values()

[6]

### Part 1

In [6]:
def amp_series(phase_settings):
    pipes = []
    for phase_setting in phase_settings:
        q = SimpleQueue()
        q.put(phase_setting)
        pipes.append(q)
    pipes.append(SimpleQueue())
    pipes[0].put(0)
    
    cpus = []
    for i in range(len(phase_settings)):
        cpu = Intcode(amp_program, input=pipes[i], output=pipes[i+1])
        cpus.append(cpu)
        
    for cpu in cpus:
        cpu.run()
        
    return pipes[-1].get()
        
amp_series(range(5))

126648

#### Part 1 Answer
Try every combination of phase settings on the amplifiers.  
**What is the highest signal that can be sent to the thrusters?**

In [7]:
def max_settings():
    highest = 0
    highest_settings = ()
    for settings in permutations(range(5)):
        result = amp_series(settings)
        if result > highest:
            highest = result
            highest_settings = settings
    return highest, highest_settings

max_settings()

(255590, (4, 1, 2, 3, 0))

---

### Part 2

In [8]:
[*range(5,10)]

[5, 6, 7, 8, 9]

In [9]:
cpu = Intcode(amp_program)
cpu.input.put(0)
cpu.input.put(0)
cpu

<Intcode pc:0/519 cycles:0 state:Paused>

In [10]:
cpu.run

<bound method Intcode.run of <Intcode pc:0/519 cycles:0 state:Paused>>

In [11]:
t = Thread(name="cpu1", target=cpu.run)

In [12]:
t.start()

In [13]:
cpu.output_values()

[65]

In [15]:
def amp_circuit(phase_settings):
    pipes = []
    for phase_setting in phase_settings:
        q = SimpleQueue()
        q.put(phase_setting)
        pipes.append(q)
    pipes.append(pipes[0])
    pipes[0].put(0)
    
    cpus = []
    for i in range(len(phase_settings)):
        cpu = Intcode(amp_program, input=pipes[i], output=pipes[i+1])
        cpu.name = f"Amp {chr(ord('A')+i)}"
        cpus.append(cpu)
        cpu.run_in_thread()
                
    for cpu in cpus:
        cpu.join()
        
    return pipes[-1].get()
        
amp_circuit(range(5,10))

42514168

#### Part 2 Answer
Try every combination of the new phase settings on the amplifier feedback loop.  
**What is the highest signal that can be sent to the thrusters?**

In [16]:
def max_settings():
    highest = 0
    highest_settings = ()
    for settings in permutations(range(5,10)):
        result = amp_circuit(settings)
        if result > highest:
            highest = result
            highest_settings = settings
    return highest, highest_settings

max_settings()

(58285150, (8, 5, 6, 9, 7))